In [67]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
from src.heikin_ashi import heikin_ashi
from src.indicators import supertrend, VWAP2, smc, wavetrend3d, atr
import pandas_ta as ta

In [66]:
auxusd = pd.read_csv("./data_directory/AUXUSD.csv")
auxusd['timestamp'] = pd.to_datetime(auxusd['timestamp'], unit='ms')
auxusd['timestamp'] = pd.to_datetime(auxusd['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
auxusd.set_index('timestamp', inplace=True)
auxusd.drop_duplicates(inplace=True)

In [137]:
usdjpy = pd.read_csv("./data_directory/USDJPY.csv")
usdjpy['timestamp'] = pd.to_datetime(usdjpy['timestamp'], unit='ms')
usdjpy['timestamp'] = pd.to_datetime(usdjpy['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
usdjpy.set_index('timestamp', inplace=True)
usdjpy.drop_duplicates(inplace=True)

In [31]:
eurgbp = pd.read_csv("./data_directory/EURGBP.csv")
eurgbp['timestamp'] = pd.to_datetime(eurgbp['timestamp'], unit='ms')
eurgbp['timestamp'] = pd.to_datetime(eurgbp['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
eurgbp.set_index('timestamp', inplace=True)
eurgbp.drop_duplicates(inplace=True)

In [3]:
forex_data = pd.read_csv("./data_directory/EURUSD_2021_2023.csv")
forex_data['timestamp'] = pd.to_datetime(forex_data['timestamp'], unit='ms')
forex_data['timestamp'] = pd.to_datetime(forex_data['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
forex_data.set_index('timestamp', inplace=True)
forex_data.drop_duplicates(inplace=True)

In [20]:
eurusd = pd.read_csv("./data_directory/EURUSD_NEW.csv")
eurusd.set_index("Date", inplace=True)
eurusd.index = pd.to_datetime(eurusd.index)
eurusd.drop_duplicates(inplace=True)

In [7]:
# Der choCH wird hier programmiert

def choCH(data, length, band):
    chocolad = pd.Series(index=data.index)
    ob = smc(data, length, band, "30T")
    prevvalue = 0
    orderblocks = []
    i = 0
    if(band == 0):
        for index, values in ob.iteritems():
            if(values > 0 and values != prevvalue):
                prevvalue = values
                give = [i, values]
                orderblocks.append(give)
            i += 1
        for i, (candle, price) in enumerate(orderblocks):
            if price > orderblocks[i-2][1] and orderblocks[i-1][1] < orderblocks[i-2][1] and orderblocks[i-2][1] < orderblocks[i-3][1] and orderblocks[i-3][1] < orderblocks[i-4][1]:
                chocolad.iloc[candle] = price
    elif(band == 1):
        for index, values in ob.iteritems():
            if(values > 0 and values != prevvalue):
                prevvalue = values
                give = [i, values]
                orderblocks.append(give)
            i += 1
        for i, (candle, price) in enumerate(orderblocks):
            if price < orderblocks[i-2][1] and orderblocks[i-1][1] > orderblocks[i-2][1] and orderblocks[i-2][1] > orderblocks[i-3][1] and orderblocks[i-3][1] > orderblocks[i-4][1]:
                chocolad.iloc[candle] = price
    return chocolad
    





/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_28160/3134265937.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  chocolad = pd.Series(index=data.index)
/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_28160/3134265937.py:10: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, values in ob.iteritems():


In [8]:

for index, value in hello.iteritems():
    if value > 0:
        print(index)

2021-06-20 20:00:00
2021-07-05 04:00:00
2021-07-22 01:00:00
2021-10-14 05:30:00
2022-01-28 15:30:00
2022-02-15 17:30:00
2022-03-07 22:00:00
2022-04-11 06:30:00
2022-04-19 15:00:00
2022-04-28 16:00:00
2022-05-09 06:30:00
2022-05-16 03:00:00
2022-06-30 19:00:00
2022-09-22 15:00:00
2022-10-24 09:30:00
2022-11-29 08:00:00
2022-12-07 16:30:00
2023-01-09 22:00:00
2023-01-19 02:00:00
2023-02-20 00:30:00
2023-02-24 01:30:00


/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_28160/2810368415.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, value in hello.iteritems():


In [199]:
def wavetrend3db(data: pd.DataFrame, cog_window, cock , timeframe, mirror):
    s_length = 1.75
    if(cock):
        dataclose = cog(data["Close"], cog_window)
    else:
        dataclose = data['Close']

    timechange = dataclose.resample(timeframe).agg({'Close': 'last'})

    signalSlow = getOscillator(timechange["Close"])

    hourly_data = pd.DataFrame(signalSlow, index=timechange.index)

    seriesSlow = hourly_data.reindex(data.index, method='pad')
    seriesSlow = s_length * seriesSlow.squeeze()
    seriesSlowMirror = -seriesSlow
    seriesSlowret = seriesSlow.ffill()
    seriesSlowretMirror = seriesSlowMirror.ffill()
    if not mirror:
        return seriesSlowret
    elif mirror:
        return seriesSlowretMirror
    

def cog(source, length):
    sum = source.rolling(length, min_periods=1).sum()
    num = 0
    for i in range(length):
        price = source.shift(i).fillna(0)
        num += price * (i + 1)
    return -(num / sum).replace([np.inf, -np.inf], np.nan).fillna(0)

def normalizeDeriv(src, quadraticMeanLength):
    derivative = src - src.shift(2)  
    quadraticMean = math.sqrt(np.nansum(np.power(derivative, 2)) / quadraticMeanLength)
    normalizedDeriv = derivative / quadraticMean  
    series = pd.Series(index=src.index, data=normalizedDeriv)
    return series

def tanh(_src):
    vals = -1 + 2/(1 + np.exp(-2*_src))
    series = pd.Series(index=_src.index, data=vals)
    return series

def dualPoleFilter(_src, _lookback):
    _omega = -99 * math.pi / (70 * _lookback)
    _alpha = np.exp(_omega)
    _beta = -np.power(_alpha, 2)
    _gamma = np.cos(_omega) * 2 * _alpha
    _delta = 1 - _gamma - _beta
    _slidingAvg = 0.5 * (_src + np.nan_to_num(_src.shift(1), nan=_src[0]))
    _filter = np.empty_like(_src)
    _filter[0] = np.nan
    
    for i in range(1, len(_src)):
        _filter[i] = (_delta * _slidingAvg[i]) + _gamma * np.nan_to_num(_filter[i-1], nan=0) + _beta * np.nan_to_num(_filter[i-2], nan=0)
    series = pd.Series(_filter, index=_src.index)
    return series

def getOscillator(data, smoothingFrequency=40, quadraticMeanLength=50):
    nDeriv = normalizeDeriv(data, quadraticMeanLength)
    hyperbolicTangent = tanh(nDeriv)
    result = dualPoleFilter(hyperbolicTangent, smoothingFrequency)
    return result

In [ ]:
indexes = argrelextrema(auxusd['High'][:1000].values, np.greater, order=20)[0]
values = auxusd['High'][:1000].iloc[indexes]

indexeslower = argrelextrema(auxusd['Low'][:1000].values, np.less, order=20)[0]
valueslower = auxusd['Low'][:1000].iloc[indexeslower]

plt.figure(figsize=(20,10))
plt.plot(auxusd['Close'][:1000])
plt.scatter(values.index, values.values, c='r')
plt.scatter(valueslower.index, valueslower.values, c='r')



In [ ]:

plt.figure(figsize=(20,10))
plt.plot(auxusd["Close"][3000:6000])
plt.plot(pivotPoints(auxusd[3000:6000], 50, 0))
plt.plot(pivotPoints(auxusd[3000:6000], 50, 1))